# This is a quick tutorial on Tensorflow based on Stanford CS 20SI

Now it's time for us to train a deep learning model on MNIST dataset

<font color = green> **This section will be a basebone for orther project (seriously, deep learning models are only different on model, cost function, optimizations, oops... (too much))**  <font>

In [2]:
import tensorflow as tf
import os                    # For path
import numpy as np           # You should know this
import time                  # For estimating running time

In [3]:
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets("MNIST_data/", one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [4]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f495b3248d0>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f495b324d50>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7f495b324bd0>)

<font color = green>** As discussed in part 3a, variable scope is important for maintaining the consistency of our program**</font> <br>

Let's see how that is going to use in practice, for this tutorial, it is highly recommended that **CS231n has been reviewed (we will use batch normalization, convolutional layer)** 

In [5]:
"""For this lesson, we will use conv net func"""
def conv_relu(input, kernel_shape, bias_shape):   
    '''Create variable for convolutional layer'''
    # Create variable named "weights".
    weights = tf.get_variable("weights", kernel_shape,
        initializer=tf.random_normal_initializer(stddev=0.5))
    # Create variable named "biases".
    biases = tf.get_variable("biases", bias_shape,
        initializer=tf.random_normal_initializer(stddev=0.5))
    
    '''Filter operation'''
    conv = tf.nn.conv2d(input, weights,
        strides=[1, 1, 1, 1], padding='SAME')
    
    '''These lines are for batch normalization layer'''
    
    mean = tf.get_variable("mean", bias_shape, initializer=tf.random_normal_initializer(stddev=0.5))
    var = tf.get_variable("var", bias_shape, initializer=tf.random_uniform_initializer())
    offset = tf.get_variable("off-set", bias_shape, initializer=tf.random_normal_initializer(stddev=0.5))
    scale = tf.get_variable("scale", bias_shape, initializer=tf.random_normal_initializer(stddev=0.5))
    
    '''Batch norm in action'''
    conv_norm = tf.nn.batch_normalization(tf.add(conv, biases), mean, var, offset, scale, variance_epsilon= 0.0001)
    
    return tf.nn.relu(conv_norm)

def fully_connected(input, hidden_shape,bias_shape):
    '''Similar...'''
    weights = tf.get_variable("weights", hidden_shape, initializer= tf.random_normal_initializer(stddev=0.5))
    biases = tf.get_variable("biases", bias_shape,initializer=tf.random_normal_initializer(stddev=0.5))
    #print weights
    mean = tf.get_variable("mean", bias_shape, initializer=tf.random_normal_initializer(stddev=0.5))
    var = tf.get_variable("var", bias_shape, initializer=tf.random_uniform_initializer())
    offset = tf.get_variable("off-set", bias_shape, initializer=tf.random_normal_initializer(stddev=0.5))
    scale = tf.get_variable("scale", bias_shape, initializer=tf.random_normal_initializer(stddev=0.5))
    fc_norm = tf.nn.batch_normalization(tf.add(tf.matmul(input, weights), biases), mean, var, offset, scale, variance_epsilon= 0.0001)
    #fc_norm = tf.contrib.layers.batch_norm(tf.add(tf.matmul(input, weights), biases),center=True, scale=True, 
    #                                      is_training=True)
    return tf.nn.relu(fc_norm)


def sofmax_out(input, hidden_shape, bias_shape):
    weights = tf.get_variable("weights", hidden_shape, initializer= tf.random_normal_initializer(stddev=0.5))
    biases = tf.get_variable("biases", bias_shape,initializer=tf.random_normal_initializer(stddev=0.5))
    return tf.add(tf.matmul(input, weights), biases)

** WHY DO WE NEED TO USE RANDOM_UNIFORM FOR INITIALIZING VARIANCE? **

In [6]:
"""We've got a dataset"""
#Step1: define input output
with tf.name_scope("data"):
    x = tf.placeholder(tf.float32, [None, 784], name= "pixels_input")
    y = tf.placeholder(tf.float32, [None, 10], name = "label")
    images = tf.reshape(x, shape=[-1, 28, 28, 1])  # <== we use images as input, not string
#Step 2: define a model
with tf.name_scope("model"):
    """Now define a CNN network to classify this:
    My proposal:
    1./ Conv Layer: 
    2./ Pooling Layer: 
    3./ Conv Layer again: 
    4./ Pool again:
    5./ Fully connected
    6./ Fully connected to output
    Each layer inside has a batch norm layer
    Nonlinearity: relu"""
    with tf.variable_scope("conv1") as scope: # <<<<<<<<<<<<We need this line to manage our variables
        relu1_norm = conv_relu(images, [5, 5, 1, 32], [32])
        #relu1_norm = relu1
        #relu1_norm = tf.layers.batch_normalization(relu1)
    with tf.variable_scope("pool1") as scope:
        pool1 = tf.nn.max_pool(relu1_norm, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1],padding='SAME')
    with tf.variable_scope("conv2") as scope:
        relu2_norm = conv_relu(pool1, [5,5,32,64], [64])
        #relu2_norm = tf.layers.batch_normalization(relu2)
        #relu2_norm = relu2
    with tf.variable_scope("pool2") as scope:
        pool2_norm = tf.nn.max_pool(relu2_norm, ksize=[1, 2, 2 ,1], strides =[1, 2, 2,1], padding = 'SAME')
        #pool2_norm = tf.layers.batch_normalization(pool2)
        #pool2_norm = pool2
        pool2_norm = tf.reshape(pool2_norm, [-1, 7 * 7 * 64]) #before feeding to fully_connected
    with tf.variable_scope("fully_connected") as scope:
        fc_norm = fully_connected(pool2_norm, [7*7*64, 1024], 1024)
        #fc_norm = fc
        #fc_norm = tf.layers.batch_normalization(fc)
    with tf.variable_scope("output_layer") as scope:
        out = sofmax_out(fc_norm, [1024, 10], 10)

In [7]:
#define loss function
with tf.name_scope("loss"):
    entropy = tf.nn.softmax_cross_entropy_with_logits(labels=y, logits=out)
    loss = tf.reduce_mean(entropy, name='entropy_loss')

In [15]:
entropy

<tf.Tensor 'loss/Reshape_2:0' shape=(?,) dtype=float32>

In [9]:
with tf.name_scope('summaries'):
    tf.summary.scalar('loss', loss)
    tf.summary.histogram('histogram_loss', loss)
    summary_op = tf.summary.merge_all()
    

In [10]:
LEARNING_RATE = 0.0001
BATCH_SIZE = 128
SKIP_STEP = 40
N_EPOCHS = 5



In [11]:
"""These line of code are important for storing the stages during our training process & init variables"""
global_step = tf.Variable(0, dtype=tf.int32, trainable=False, name='global_step')
init = tf.global_variables_initializer() 

In [12]:
with tf.name_scope("Optimizer") as scope:
    optimizer = tf.train.AdamOptimizer(LEARNING_RATE).minimize(loss
                                        ,global_step=global_step)

<font color = red> **Small note :** </font> <br>
<font color = blue>
- Our optimizer need variable global_step, this is because when we interrupt our training (or our training has finished) and we train again, optimizer needs to know where it has left in order to either skip training or continue <br>
- global_step, as we shall see, will be updated in training as well
</font>

In [13]:
optimizer

<tf.Operation 'Optimizer/Adam' type=AssignAdd>

In [14]:
with tf.Session() as sess:
    #sess.run(init) #init all variables
    sess.run(tf.global_variables_initializer())
    
    
    saver = tf.train.Saver()
    writer = tf.summary.FileWriter('./graphs/convnet', sess.graph)  #Skip this for now
    ckpt = tf.train.get_checkpoint_state(os.path.dirname('checkpoints/convnet_mnist/checkpoint'))
    
    # if that checkpoint exists, restore from checkpoint
    if ckpt and ckpt.model_checkpoint_path:
        saver.restore(sess, ckpt.model_checkpoint_path)
    initial_step = global_step.eval()
    
    start_time = time.time()
    n_batches = int(mnist.train.num_examples / BATCH_SIZE)

    total_loss = 0.0
    for index in range(initial_step, n_batches * N_EPOCHS): # train the model n_epochs times
        X_batch, Y_batch = mnist.train.next_batch(BATCH_SIZE)
        
        _, loss_batch, summary = sess.run([optimizer, loss, summary_op], 
                                feed_dict={x: X_batch, y:Y_batch}) 
        
        writer.add_summary(summary, global_step=index) # '''Update the index here'''
        
        total_loss += loss_batch
        if (index + 1) % SKIP_STEP == 0:
            print('Average loss at step {}: {:5.1f}'.format(index + 1, total_loss / SKIP_STEP))
            total_loss = 0.0
            saver.save(sess, 'checkpoints/convnet_mnist/mnist-convnet', index)
    
    print("Optimization Finished!") # should be around 0.35 after 25 epochs
    print("Total time: {0} seconds".format(time.time() - start_time))
    
    # test the model
    n_batches = int(mnist.test.num_examples/BATCH_SIZE)
    total_correct_preds = 0
    for i in range(n_batches):
        X_batch, Y_batch = mnist.test.next_batch(BATCH_SIZE)
        loss_batch, logits_batch = sess.run([loss, out], 
                                        feed_dict={x: X_batch, y:Y_batch}) 
        preds = tf.nn.softmax(logits_batch)
        correct_preds = tf.equal(tf.argmax(preds, 1), tf.argmax(Y_batch, 1))
        accuracy = tf.reduce_sum(tf.cast(correct_preds, tf.float32))
        total_correct_preds += sess.run(accuracy)   
    
    print("Accuracy {0}".format(total_correct_preds/mnist.test.num_examples))
    

INFO:tensorflow:Restoring parameters from checkpoints/convnet_mnist/mnist-convnet-2119
Optimization Finished!
Total time: 19.1182291508 seconds
Accuracy 0.9137


** Hurray!! Done!!**